In [1]:
import pandas as pd
import glob
import os

# ====================================================
# [설정] 통합할 CSV 파일들이 있는 폴더
# ====================================================
# 예: .../naver_news/018_이데일리  또는  .../naver_news/combined
target_folder = r"C:\ai\lecNote\10_1stTeamProject\naver_news\test"

# ====================================================
# 1. 파일 목록 확인
# ====================================================
file_pattern = os.path.join(target_folder, "*.csv")
all_files = glob.glob(file_pattern)

# 결과 파일(merged_, .pkl) 제외
all_files = [f for f in all_files if "merged_" not in f and ".pkl" not in f]

print(f"📂 폴더 위치: {target_folder}")
print(f"🔎 발견된 CSV 파일: {len(all_files)}개")

if len(all_files) == 0:
    print("❌ 파일을 하나도 못 찾았습니다! 경로를 다시 확인해 주세요.")
else:
    # ====================================================
    # 2. 파일 순회 및 병합
    # ====================================================
    df_list = []
    collected_dates = []

    print("\n🚀 병합 작업을 시작합니다... (스마트 감지 모드)")

    for i, filename in enumerate(all_files):
        base_name = os.path.basename(filename)
        name_no_ext = os.path.splitext(base_name)[0]
        
        current_oid = "unknown"
        current_date = None

        # --- OID/날짜 파싱 로직 (구버전/신버전 호환) ---
        try:
            # Case A: oid=018&date=20251220
            if "oid=" in name_no_ext and "date=" in name_no_ext:
                parts = name_no_ext.split('&')
                for part in parts:
                    if "oid=" in part:
                        current_oid = part.replace("oid=", "")
                    elif "date=" in part:
                        current_date = part.replace("date=", "")
            # Case B: 018-20251220
            elif "-" in name_no_ext:
                parts = name_no_ext.split('-')
                if len(parts) >= 2:
                    current_oid = parts[0]
                    current_date = parts[1]
            
            if current_date:
                collected_dates.append(current_date)
        except:
            pass
        # ---------------------------------------------

        try:
            # 파일 읽기
            try:
                temp_df = pd.read_csv(filename, encoding='utf-8-sig')
            except UnicodeDecodeError:
                temp_df = pd.read_csv(filename, encoding='cp949')
            
            if temp_df.empty: continue

            # ★ 일단 OID 컬럼을 추가합니다 (나중에 판단해서 지울지 결정)
            temp_df['OID'] = current_oid
            df_list.append(temp_df)

        except Exception as e:
            print(f"   ❌ 읽기 에러: {base_name} -> {e}")

    # ====================================================
    # 3. 통합 및 스마트 처리 (여기가 핵심!)
    # ====================================================
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"\n📊 [1차 통합] 총 {len(combined_df)}건 합침 완료.")

        # 날짜 변환
        if '날짜' in combined_df.columns:
            combined_df['날짜'] = pd.to_datetime(combined_df['날짜'], errors='coerce')
        
        # ----------------------------------------------------------
        # ★ [스마트 로직] OID 개수에 따른 처리 분기
        # ----------------------------------------------------------
        unique_oids = combined_df['OID'].unique()
        unique_count = len(unique_oids)
        
        # 날짜 범위 문자열 생성
        if collected_dates:
            collected_dates.sort()
            date_range = f"{collected_dates[0]}-{collected_dates[-1]}"
        else:
            date_range = "nodate"

        final_filename = ""
        
        if unique_count == 1:
            # [상황 1] 단일 언론사 (Single OID)
            target_oid = unique_oids[0]
            print(f"   💡 단일 언론사 감지됨 ({target_oid}) -> 'OID' 컬럼 삭제 및 파일명 최적화")
            
            # 1) OID 컬럼 삭제 (불필요함)
            combined_df.drop(columns=['OID'], inplace=True)
            
            # 2) 파일명: 009_20251201-20251222.pkl
            final_filename = f"{target_oid}_{date_range}.pkl"
            
        else:
            # [상황 2] 다중 언론사 (Multi OID)
            print(f"   💡 {unique_count}개 언론사 혼합 감지 -> 'OID' 컬럼 유지")
            
            # 1) OID 컬럼 유지 (필수함)
            # 날짜와 OID 기준으로 정렬
            if '날짜' in combined_df.columns:
                combined_df = combined_df.sort_values(by=['날짜', 'OID']).reset_index(drop=True)

            # 2) 파일명: oids_20251201-20251222.pkl
            final_filename = f"oids_{date_range}.pkl"

        # ====================================================
        # 4. 저장
        # ====================================================
        save_path = os.path.join(target_folder, final_filename)
        combined_df.to_pickle(save_path)
        
        print(f"\n✅ 최종 저장 완료: {save_path}")
        print("[데이터 미리보기]")
        print(combined_df.head())

    else:
        print("\n❌ 합칠 데이터가 없습니다.")

📂 폴더 위치: C:\ai\lecNote\10_1stTeamProject\naver_news\test
🔎 발견된 CSV 파일: 231개

🚀 병합 작업을 시작합니다... (스마트 감지 모드)

📊 [1차 통합] 총 81368건 합침 완료.
   💡 11개 언론사 혼합 감지 -> 'OID' 컬럼 유지

✅ 최종 저장 완료: C:\ai\lecNote\10_1stTeamProject\naver_news\test\oids_20251201-20251221.pkl
[데이터 미리보기]
                   날짜                                        제목  \
0 2025-12-01 00:01:00  "살게 되면 소주나 한 잔"…침몰 어선 속 선장의 하직 인사[그해 오늘]   
1 2025-12-01 00:06:00        [사설] 대미 투자로 기업들 해외 이탈하는데 법인세까지 올리나   
2 2025-12-01 00:06:00      [사설] 꼬일대로 꼬인 ‘지하철 무임승차’, 해결책 마련 서둘러야   
3 2025-12-01 00:06:00    [사설] 이번엔 쿠팡 고객 정보 다 털려…‘AI시대 보안’이 불안하다   
4 2025-12-01 00:31:00   "피 같은 감귤 3톤이 몽땅 사라졌다"…수확철 앞둔 제주서 무슨 일이?   

                                                  본문  \
0  [이데일리 이로원 기자] 2014년 12월 1일, 사조산업 명태잡이 어선 ‘501오...   
1  [서울경제]  여야가 대기업 법인세율을 올리기로 가닥을 잡은 모양새다. 주요국들이 ...   
2  [서울경제]  전국 지하철의 재정 손실이 눈덩이처럼 불어나고 있는 가운데 이에 대한...   
3  [서울경제]  국내 e커머스 1위 기업인 쿠팡에서 3370만 건에 달하는 대규모 개...   
4  [서울경제]  감귤 수확 철을 맞은 제주에서 농산물 절도 범죄가 급증하고 있어 주의.